In [1]:
# extract data from api 

In [2]:
# FILE : extract_from_api.py

In [3]:
import luigi
from datetime import (date, datetime)
import pickle
import re
from sodapy import Socrata
import boto3

In [7]:
import joblib

In [4]:
## This should be en utils"
def get_client(token):
    return Socrata("data.cityofchicago.org", token)

def ingesta_inicial(client, limit=300000):
    return client.get("4ijn-s7e5", limit=limit)

def ingesta_consecutiva(client, soql_query):
    return client.get("4ijn-s7e5", where=soql_query)

In [5]:
## Parameters
## AWS parameters
bucket_name = "data-product-architecture-equipo-9"

hist_ingest_path = "ingestion/initial/"
hist_dat_prefix = "historic-inspections-"

cont_ingest_path = "ingestion/consecutive/"
cont_dat_prefix = "consecutive-inspections-"

## Naming files
today_info = date.today().strftime('%Y-%m-%d')



In [6]:
class APIDataIngestion(luigi.Task):
    
    ## luigi. parameter... /// se lo envia el task siguiente ... initial consecutive
    ingest_type = luigi.Parameter()
    
    def output(self):
        # guardamos en archivo local para que qeude registro de que se ejecuto el task 
        return luigi.local_target.LocalTarget('/luigi_tmp_files/ingesta_tmp.pkl')
    
    def run(self):
        ## Getting client to download data with API
        client = get_client(token)
        
        if ingest_type=='initial':
            ingesta = pickle.dumps(ingesta_inicial(client))
         
        elif ingest_type=='consecutive':
            
            ## Finding most recent date in consecutive pickles

            #### Getting list with pickles stored in s3 consecutive
            objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=cont_ingest_path)['Contents']

            #### Regular expression to isolate date in string
            regex = str(cont_dat_prefix) + "(.*).pkl"

            #### List of all dates in consecutive pickles
            pkl_dates = [datetime.strptime(re.search(regex, obj["Key"]).group(1), '%Y-%m-%d') for obj in objects if cont_dat_prefix in obj["Key"]]

            #### Consecutive pickle most recent date
            pkl_mrd = datetime.strftime(max(pkl_dates), '%Y-%m-%d')

            ## Building query to download data of interest
            soql_query = "inspection_date >= '{}'".format(pkl_mrd)

            ingesta = pickle.dumps(ingesta_consecutiva(client, soql_query))
        
        with self.output().open('w') as output_file:
            output_file.write("test,luigi,s3")
            
        


In [ ]:
luigi 

In [3]:
## Guardar en S3

In [ ]:
#imports for this .py 
import luigi
import luigi.contrib.s3
import boto3
import os

In [4]:
!pwd

/Users/sansansansan/repos_itam/dpa_2021/notebooks


In [11]:
import sys
sys.path.append("../")

In [13]:
#local imports: 
#from extract_from_api import APIDataIngestion
from src.etl.ingesta_almacenamiento import get_s3_resource

In [ ]:
# existe un bug con bot3 y luigi para pasar las credenciales
# necesitas enviar el parámetro AWS_PROFILE e indicar el profile
# con el que quieres que se corra
# PYTHONPATH='.' AWS_PROFILE=mge luigi --module ex3_luigi S3Task --local-scheduler ...


class S3Task(luigi.Task):
    
    
    ## read '/luigi_tmp_files/ingesta_tmp.pkl'
    
    
    bucket = luigi.Parameter()
    root_path = luigi.Parameter()
    ingest_type = luigi.Parameter() #initial or consecutive
    year = luigi.Parameter()
    month = luigi.Parameter()
    day=luigi.Parameter()
    
    def requires(self):
        return APIDataIngestion(self)

    
    def output(self):

        output_path = "s3://{}/{}/{}/{}/YEAR={}/MONTH={}/ingesta.pkl".\
        format(self.bucket,
        self.root_path,
        self.ingest_type,
        self.year,
        str(self.month),
        self.day)
        
        
    def run(self):
        ## read file... 
        
        s3 = get_s3_resource()
        #ses = boto3.session.Session(profile_name='mge', region_name='us-west-2')
        #s3_resource = ses.resource('s3')
              
        ingesta=pickle.load('/luigi_tmp_files/ingesta_tmp.pkl')

        with self.output().open('w') as output_file:
            output_file.write("test,luigi,s3")
            
        with self.output().open('w') as s3_f:
            joblib.dump(ingesta, s3_f)


        return luigi.contrib.s3.S3Target(path=output_path)

### call from command line
bucket = data-product-architecture-equipo-9
root_path = ingestion
ingest_type = #initial or consecutive
year = 2021
month = 03
#day = 12
    